### Install Dependincies

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade accelerate
!pip install torch torchvision torchaudio

In [ ]:
!pip show transformers
!pip show accelerate
!pip show torchvision
!pip show torchaudio

### Connect to drive to save or load models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !ls '/content/drive/My Drive/' | grep '\.zip$'

### Downloading the Dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Download the dataset
!gdown --id 1NdvIddoyYy2idsAWxJ8lodKfD-PZhmyL

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1NdvIddoyYy2idsAWxJ8lodKfD-PZhmyL
To: /content/in_domain_train.tsv
100% 429k/429k [00:00<00:00, 7.53MB/s]


In [3]:
# Read into a pandas dataframe
df = pd.read_csv("in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])[['label', 'sentence']]
df.head()

,label,sentence
0,1,"Our friends won't buy this analysis, let alone..."
1,1,One more pseudo generalization and I'm giving up.
2,1,One more pseudo generalization or I'm giving up.
3,1,"The more we study verbs, the crazier they get."
4,1,Day by day the facts are getting murkier.


In [4]:
df.label.value_counts()

label
1    6023
0    2528
Name: count, dtype: int64

In [11]:
import random
import nltk
from nltk.corpus import brown
nltk.download('brown')

def generate_sentences(n):
    sentences = brown.sents()
    grammatically_correct_sentences = [' '.join(sentence) for sentence in sentences]
    selected_sentences = random.sample(grammatically_correct_sentences, n)
    return selected_sentences

num_sentences = 3495
correct_sentences = generate_sentences(num_sentences)

# with open('correct_sentences.txt', 'w') as f:
#     for sentence in correct_sentences:
#         f.write(sentence + '\n')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [12]:
correct_sentences[:10]

["That was why he hadn't been able to finish the porch .",
 "You've probably given a lot of Christmas-season thought to the guns in your rack , but it's not easy to decide on a new one .",
 'The achievements which dispelled our fears of the cosmos took place three centuries ago .',
 "It probably wasn't hard for him to keep his nose to the grindstone with nothing but Edythe to come home to .",
 "Meredith's voice was always deep , with rough bass notes in it ; ;",
 "Soon as the Burnsides moved on , he'd lead Rex down by the river ; ;",
 "I can hardly wait '' .",
 'The English have managed to hold onto their madrigal tradition better than anyone else .',
 "Continue on the Via D. Baullari to the Corso Vittorio Emanuele , then turn right for a couple of hundred yards to the Church of Sant' Andrea Della Valle .",
 "It was in the spring of the year when he took to his bed and Tessie and Alfred found out that they didn't know each other ."]

In [13]:
len(correct_sentences)

3495

In [14]:
df_correct = pd.DataFrame()

In [17]:
df_correct['label'] = [0 for i in range(3495)]
df_correct['sentence'] = correct_sentences

In [18]:
df_correct.head()

,label,sentence
0,0,That was why he hadn't been able to finish the...
1,0,You've probably given a lot of Christmas-seaso...
2,0,The achievements which dispelled our fears of ...
3,0,It probably wasn't hard for him to keep his no...
4,0,"Meredith's voice was always deep , with rough ..."


In [19]:
df_correct.label.value_counts()

label
0    3495
Name: count, dtype: int64

In [20]:
balanced_df = pd.concat([df, df_correct])

In [21]:
balanced_df.head()

,label,sentence
0,1,"Our friends won't buy this analysis, let alone..."
1,1,One more pseudo generalization and I'm giving up.
2,1,One more pseudo generalization or I'm giving up.
3,1,"The more we study verbs, the crazier they get."
4,1,Day by day the facts are getting murkier.


In [22]:
balanced_df.label.value_counts()

label
1    6023
0    6023
Name: count, dtype: int64

### Model Building, Training and Evaluation

In [23]:
import torch
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [24]:
X = list(balanced_df.sentence)
y = list(balanced_df.label)

In [25]:
train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [26]:
set(y)

{0, 1}

In [27]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)

In [28]:
checkpoint = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
model = DistilBertForSequenceClassification.from_pretrained(checkpoint, num_labels=len(set(y)), problem_type="single_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# Lets build custom dataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
      text = str(self.texts[idx])
      label = torch.tensor(self.labels[idx])

      encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=self.max_len, return_tensors='pt')
      # encoding = self.tokenizer(text, truncation=True, padding=True, return_tensors='pt')

      return {
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': label,
          # 'loss': torch.tensor(0.0),   # Placeholder for loss calculation
      }

In [30]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def binary_classification_metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = (probs.numpy() >= threshold).astype(int)
    y_true = labels

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, probs.numpy())

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc
    }

    return metrics

In [32]:
import os

os.makedirs('/content/results', exist_ok=True)
os.makedirs('/content/logs', exist_ok=True)

In [33]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define early stopping callback
early_stopping = EarlyStoppingCallback(early_stopping_patience = 4)  # Adjust patience as needed


# Define custom Trainer class with modified loss function
class TrainerWithCustomLoss(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


# Define evaluation metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


# Define TrainingArguments
args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    output_dir='/content/results',
    num_train_epochs=8,
    save_steps=300,
    save_total_limit=2,
    warmup_steps=150,
    weight_decay=1e-4,
    load_best_model_at_end=True,
    evaluation_strategy="steps",
    eval_steps=300,
    logging_dir='/content/logs',
    learning_rate=1e-5,  # Adjust the learning rate as needed
)


# Initialize Trainer
trainer = TrainerWithCustomLoss(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

{'eval_loss': 0.39764708280563354,
 'eval_accuracy': 0.8195020746887967,
 'eval_precision': 0.7662337662337663,
 'eval_recall': 0.9233937397034596,
 'eval_f1': 0.8375046694060515,
 'eval_runtime': 9.6073,
 'eval_samples_per_second': 250.852,
 'eval_steps_per_second': 31.435,
 'epoch': 1.991701244813278}

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/distilbert-tokenizer')

('/content/drive/MyDrive/distilbert-tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/distilbert-tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/distilbert-tokenizer/vocab.txt',
 '/content/drive/MyDrive/distilbert-tokenizer/added_tokens.json')

In [ ]:
model.save_pretrained("/content/drive/My Drive/distilbert-finetuned-binary-classifier-v2")

In [ ]:
!zip -r /content/model.zip /content/distilbert-finetuned-binary-classifier

#### Make predictions

In [ ]:
text = "give him her sududu."

encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

In [ ]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.3862,  0.3495]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
import numpy as np

# Get the index of the maximum probability
pred_index = np.argmax(outputs)

# Assuming the classes are named 'Class 0' and 'Class 1'
pred_class = 'Class 0' if pred_index == 0 else 'Class 1'

print(pred_class)

Class 0


In [ ]:
import torch
import numpy as np

def predict_class(text, tokenizer, model):
    # Tokenize the text
    encoding = tokenizer(text, return_tensors='pt')

    # Move the input tensors to the model's device
    encoding.to(model.device)

    # Get the model's prediction
    with torch.no_grad():
        outputs = model(**encoding)

    # Get the index of the maximum probability
    pred_index = np.argmax(outputs.logits.cpu().numpy())

    # Assuming the classes are named 'Class 0' and 'Class 1'
    pred_class = f'"{text}" is grammatically CORRECT' if pred_index == 0 else f'"{text}" is grammatically INCORRECT'

    return pred_class

In [ ]:
# Example usage
text = "Thank you, see you soon"
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

Thank you, see you soon is grammatically CORRECT


In [ ]:
"Give up when you know the war is ended"
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

Thank you, see you soon is grammatically CORRECT


In [ ]:
"Give up are fine and goods"
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

Thank you, see you soon is grammatically CORRECT


In [ ]:
"Our friends won't buy this analysis, let alone the next one we propose."
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

Thank you, see you soon is grammatically CORRECT


#### load the model

In [ ]:
# !unzip '/content/drive/My Drive/distilbert-finetuned-binary-classifier.zip'

unzip:  cannot find or open /content/drive/My Drive/distilbert-finetuned-binary-classifier.zip, /content/drive/My Drive/distilbert-finetuned-binary-classifier.zip.zip or /content/drive/My Drive/distilbert-finetuned-binary-classifier.zip.ZIP.


In [ ]:
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification

# Load the model
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/distilbert-finetuned-binary-classifier-v2', problem_type="single_label_classification")

# Make sure to set the model to evaluation mode
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
# Initialize Trainer
trainer = TrainerWithCustomLoss(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.39764708280563354,
 'eval_accuracy': 0.8195020746887967,
 'eval_precision': 0.7662337662337663,
 'eval_recall': 0.9233937397034596,
 'eval_f1': 0.8375046694060515,
 'eval_runtime': 10.3402,
 'eval_samples_per_second': 233.071,
 'eval_steps_per_second': 29.206}

In [ ]:
text = "Thank you, see you soon"
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

"Thank you, see you soon" is grammatically CORRECT


In [ ]:
text = "me and ali is in the pool alone"
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

"me and ali is in the pool alone" is grammatically CORRECT


In [ ]:
text = "me is ali is in the pool alone."
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

"me is ali is in the pool alone." is grammatically CORRECT


In [ ]:
text = "do it once again and less se his cow in the road."
predicted_class = predict_class(text, tokenizer, trainer.model)
print(predicted_class)

"do it once again and less se his cow in the road." is grammatically CORRECT
